In [ ]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@nightly git+https://github.com/unslothai/unsloth-zoo.git

Found existing installation: unsloth 2026.1.3
Uninstalling unsloth-2026.1.3:
  Successfully uninstalled unsloth-2026.1.3
  Cloning https://github.com/unslothai/unsloth.git (to revision nightly) to /tmp/pip-req-build-_rxd4g11
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-_rxd4g11
  Running command git checkout -b nightly --track origin/nightly
  Switched to a new branch 'nightly'
  Branch 'nightly' set up to track remote branch 'nightly' from 'origin'.
  Resolved https://github.com/unslothai/unsloth.git to commit 8452e2ae376f4ce65e2f333a3cd9be22d3d1deae
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/unslothai/unsloth-zoo.git to /tmp/pip-req-build-ib6venj9
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth-zoo.git /tmp/pip-req-build-ib6venj9
  Resolved https://gith

In [ ]:
import os
import torch
import pandas as pd
from unsloth import FastLanguageModel
from datasets import Dataset, load_dataset, concatenate_datasets
from trl import SFTTrainer
from transformers import TrainingArguments
from huggingface_hub import login

from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')
login(HF_TOKEN)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model_name = 'unsloth/Qwen3-4B-unsloth-bnb-4bit'
repo_name = 'VyDat/qwen3-4b-bnb-4bit'

data_path = 'VyDat/vsl-data'
data_augment_path = 'VyDat/Copus-Vie-VSL-10K'
pub_data_path = '5CD-AI/Vietnamese-Multi-turn-Chat-Alpaca'

max_seq_length=2048
dtype=None

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
)

==((====))==  Unsloth 2026.1.3: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2026.1.3 patched 36 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
def format_data_hf(data_path):
    print(f"Đang tải dataset từ: {data_path}")
    try:
        raw_dataset = load_dataset(data_path, split="train")
    except Exception as e:
        print(f"Lỗi khi tải dataset từ Hub ({data_path}): {e}")
        raise

    def formatting_function(examples):
        texts = []
        for messages in examples["messages"]:
            # yêu cầu tối thiểu có user + assistant
            if len(messages) < 2:
                print(f"Warning! Tin nhắn không hợp lệ: {messages}")
                continue

            # dùng trực tiếp messages, không thêm system
            text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False
            )
            texts.append(text)

        return {"text": texts}

    if "messages" not in raw_dataset.column_names:
        raise ValueError("Cột 'messages' không tồn tại trong dataset")

    formatted_dataset = raw_dataset.map(
        formatting_function,
        batched=True,
        remove_columns=raw_dataset.column_names,
        desc="Applying chat template (no system instruction)",
        num_proc=4
    )

    print(f"Thành công! Số lượng mẫu: {len(formatted_dataset)}")
    return formatted_dataset


vsl_dataset = format_data_hf(data_path)
data_augment = format_data_hf(data_augment_path)


Đang tải dataset từ: VyDat/vsl-data
Thành công! Số lượng mẫu: 7544
Đang tải dataset từ: VyDat/Copus-Vie-VSL-10K
Thành công! Số lượng mẫu: 10000


In [ ]:
print(vsl_dataset[0]["text"])
print(data_augment[0]["text"])

<|im_start|>user
Tớ đang tò mò không biết, câu hỏi 'Bạn tên gì?' khi chuyển sang Ngôn ngữ Ký hiệu Việt Nam thì có phải chỉ đơn giản là ký hiệu từng từ một y hệt tiếng Việt, hay mình cần dùng cấu trúc ký hiệu đặc trưng nào khác không nhỉ?<|im_end|>
<|im_start|>assistant
<think>

</think>

À, với câu 'Bạn tên gì?' trong VSL, bạn cứ yên tâm là nó được biểu diễn khá trực tiếp đó! Chính xác là bạn sẽ ký hiệu theo thứ tự 'Bạn - Tên - Gì' như cách nói thông thường vậy.<|im_end|>

<|im_start|>user
Làm sao để ra dấu câu Tôi 19 tuổi .?<|im_end|>
<|im_start|>assistant
<think>

</think>

Tôi tuổi 19 .<|im_end|>



In [ ]:
def normalize_messages(messages):
    normalized = []
    for m in messages:
        if m["from"] == "human":
            role = "user"
        elif m["from"] == "gpt":
            role = "assistant"
        else:
            continue

        normalized.append({
            "role": role,
            "content": m["value"]
        })
    return normalized


def format_public_ds(data_path):
    print(f"Đang tải dataset từ: {data_path}")
    try:
        raw_dataset = load_dataset(data_path, split="train")
    except Exception as e:
        print(f"Lỗi khi tải dataset từ Hub ({data_path}): {e}")
        raise

    if "conversations" not in raw_dataset.column_names:
        raise ValueError("Cột 'conversations' không tồn tại trong dataset")

    def formatting_function(examples):
        texts = []

        for messages in examples["conversations"]:
            if len(messages) < 2:
                continue

            messages = normalize_messages(messages)

            if len(messages) < 2:
                continue

            text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False
            )
            texts.append(text)

        return {"text": texts}

    formatted_dataset = raw_dataset.map(
        formatting_function,
        batched=True,
        remove_columns=raw_dataset.column_names,
        desc="Applying chat template (alpaca → HF)",
        num_proc=4
    )

    print(f"Thành công! Số lượng mẫu: {len(formatted_dataset)}")
    return formatted_dataset

public_data = format_public_ds(pub_data_path)
print(public_data[0]["text"])


Đang tải dataset từ: 5CD-AI/Vietnamese-Multi-turn-Chat-Alpaca
Thành công! Số lượng mẫu: 12697
<|im_start|>user
Hãy chỉnh sửa câu này để ngắn gọn hơn mà không mất đi ý nghĩa: "Trận đấu là một thất bại nặng nề mặc dù thực tế là cả đội đã tập luyện trong nhiều tuần."<|im_end|>
<|im_start|>assistant
Nhiều tuần huấn luyện của đội đã dẫn đến một thất bại nặng nề.<|im_end|>
<|im_start|>user
Bạn có thể đề xuất một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của họ trong trận đấu tiếp theo không?<|im_end|>
<|im_start|>assistant
Chắc chắn, đây là một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của mình trong trận đấu tiếp theo: 1. Phân tích trận đấu trước bằng cách xem lại cảnh quay trận đấu để xác định điểm yếu và các lĩnh vực cần cải thiện. 2. Tăng cường độ và sự tập trung của các buổi tập để đảm bảo mọi cầu thủ đều được chuẩn bị đầy đủ về thể chất và tinh thần. 3. Luyện tập những kỹ năng cụ thể cần cải thiện, chẳng hạn như sút bóng hoặc chuyền bóng chính xác. 4

In [ ]:
combined_data = concatenate_datasets([vsl_dataset, data_augment, public_data])
print(len(combined_data))

# Chia 90% train và 10% validation
split_dataset = combined_data.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print(f"Số mẫu train (90%): {len(train_dataset)}")
print(f"Số mẫu validation (10%): {len(eval_dataset)}")

30241
Số mẫu train (90%): 27216
Số mẫu validation (10%): 3025


In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=4,
    packing=True,
    args=TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        num_train_epochs=2,
        learning_rate=3e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=100,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs",
        save_strategy="steps",
        save_steps=200,
        eval_strategy="steps",
        eval_steps=200,
        save_total_limit=3,
        report_to="none",
    ),
)

In [ ]:
print("Start Trainning.....")
torch.cuda.empty_cache()

trainer_stats = trainer.train()

print(f"Trainning successfull, Total Loss: {trainer_stats.training_loss}")

The model is already on multiple devices. Skipping the move to device specified in `args`.


Start Trainning.....


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 27,216 | Num Epochs = 2 | Total steps = 3,402
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)


Step,Training Loss,Validation Loss
200,1.146100,1.129898
400,1.121100,1.091131
600,1.056700,1.065207
800,1.049000,1.049321
1000,1.045900,1.031820
1200,1.036700,1.016400
1400,1.009100,1.008167
1600,1.004200,0.997230
1800,0.852700,0.998452
2000,0.848900,0.994870


Unsloth: Not an error, but Qwen3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Trainning successfull, Total Loss: 0.95956377906704


In [ ]:
print("Merging LoRA adapters into base model...")
model = model.merge_and_unload()

model.save_pretrained(repo_name)
tokenizer.save_pretrained(repo_name)

try:
    model.push_to_hub(repo_name, use_temp_dir=True)
    tokenizer.push_to_hub(repo_name, use_temp_dir=True)
    print(f"Training hoàn thành! Model đã được đẩy lên repo: https://huggingface.co/{repo_name}")
except Exception as e:
    print(f"Đã xảy ra lỗi khi đẩy model lên Hub: {e}")
    print(f"Model đã được lưu local tại './{repo_name}'")

Merging LoRA adapters into base model...


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


README.md:   0%|          | 0.00/553 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

...

README.md:   0%|          | 0.00/559 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mp7z1ly0ea/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

Training hoàn thành! Model đã được đẩy lên repo: https://huggingface.co/VyDat/qwen3-4b-bnb-4bit
